<a href="https://colab.research.google.com/github/promptmule4real/demo/blob/main/promptmule_reporting_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PromptMule API Demo Suite

Welcome to the PromptMule API Demo Suite! This code offers you a comprehensive way to test your interactions with the PromptMule API by generating various prompts, analyzing the semantic scores of responses, and compiling detailed reports on the tokens used and saved through caching. Let's walk through what this code does and how you can make the most of it.

#### What is this Code?

This script facilitates your interaction with the PromptMule API, a platform that offers optimized API requests to different AI models. By using this demo suite, you can:
- Authenticate and get a token for API interactions.
- Generate or fetch your user-specific API key.
- Send a series of prompt requests to the API, with varying semantic similarities and max response counts.
- Track and print the detailed logs of each API response (if verbose mode is enabled).
- Compile and display a structured report of the prompt responses and cache savings.

#### How to Use:

1. **Start the Script**: Run the script in a Python environment.
2. **Enter User Details**: You'll be prompted to enter your username, password, and the name of your application.
3. **Choose Verbose Mode**: If you'd like to see detailed logs for each API call/response, choose "yes" for verbose mode. If you prefer a silent mode with minimal output, select "no".
4. **Provide Cost-saving Strategy**: Input the name of your cost-saving strategy when prompted. This will be used to generate prompts for testing.
5. **Review the Reports**: After all the prompts have been sent and responses received, you'll get:
    - A detailed report of the prompt responses.
    - A summary of your usage.
    - A report on the savings achieved using caching.
6. **Profile Deletion (Optional)**: At the end, you'll be given the option to delete your user profile. Only choose this if you're sure, as this action cannot be undone.

#### Additional Information:
For detailed documentation, further guidance, or any other queries related to the PromptMule API, please visit [PromptMule](https://www.promptmule.com/).

Happy testing!

In [ ]:
"""
Promptmule Semantic Cache Reporting Guide
Date: October 12, 2023
Purpose: To provide guidance and best practices for application developers looking to save money
during the development and production phases of building generative AI-based applications with OpenAI.

Author: the friendly folks at Promptmule
"""

import pandas as pd
import requests
import json
import random

# The Endpoints: Constants for API interactions
ENDPOINT = 'https://api.promptmule.com/'
LOGIN = 'login'
KEY_GEN = 'api-keys'
PROMPT = 'prompt'
LIST_API_KEYS = 'api-keys/'
DELETE = 'profile/'

# User-specific details
# These are what you'll use to define your login and app for Promptmule to know it's you
USERNAME = "YOUR_USER_NAME"
PASSWORD = "YOUR_PASSWORD"
APPNAME = "YOUR_APPNAME"

# simple prompt template to give some test prompts for this demo
# example: USERS_MESSAGE = "Please provide a cost-saving strategy for implementing AI applications while we establish the API account: "
#
USERS_MESSAGE = "Please provide a cost-saving strategy for implementing AI applications while we establish the API account: "

# Headers for API interactions
headers = {'Content-Type': 'application/json'}

# --- FUNCTIONS ---

def login_to_promptmule():
    """
    Log into PromptMule service and retrieve authentication token.
    """
    data = {"username": USERNAME, "password": PASSWORD}
    response = requests.post(ENDPOINT + LOGIN, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["token"]
    else:
        print(f"Login failed: {response.status_code}")
        print(json.dumps(response.json(), indent=4))
        exit()

def generate_or_fetch_api_key():
    """
    Attempt to generate a new API key for the user's app or fetch an existing one.
    """
    data = {"app-name": APPNAME}
    response = requests.post(ENDPOINT + KEY_GEN, headers=headers, json=data)

    if response.status_code == 200 and "api-key" in response.json():
        return response.json()["api-key"]
    else:
        # If generating a new one fails, try to fetch an existing API key
        response = requests.get(ENDPOINT + LIST_API_KEYS, headers=headers)
        if response.status_code == 200:
            for key_data in response.json().get("api-keys", []):
                if key_data.get("app-name") == APPNAME:
                    return key_data.get("api-key")
    print("Error fetching or generating API keys.")
    exit()

def generate_prompt(users_message, variation):
    """
    Formulate a prompt message given a user's message and a variation number.
    """
    base_prompt = "You are a developer aiming to save money using OpenAI. Share an implementation tip related to the strategy: "
    return f"{base_prompt}{users_message}. Variation: {variation}"

def track_semantic_scores(response_json):
    """
    Extract semantic scores and associated data from the API's response.
    """
    return [(choice['prompt-id'], choice.get('score'), choice['message']['content'], choice['index']) for choice in response_json['choices']]

def send_prompt_requests(users_message, semantic_similarity, max_response_quantity):
    """
    Send a series of prompt requests to the API based on the specified parameters.
    """
    for variation in range(1, int(prompt_variations * 0.9) + 1):  # Use 90% of the available variations
        api_call_body = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": generate_prompt(users_message, variation)}],
            "max_tokens": "50",
            "temperature": "0.99",
            "user": USERNAME,
            "api": "openai", # this is specific to Promptmule, and selects the LLM you'd like to prompt
            "semantic": str(semantic_similarity), # this determines the percentage prompt similarity that is acceptable to return from the cache for this call
            "sem_num": str(max_response_quantity) # this determines the max number of prompt/response pairs to return if they are greater than "semantic" percentage match
        }

        response = requests.post(ENDPOINT + PROMPT, headers={'x-api-key': api_key, 'Content-Type': 'application/json'}, json=api_call_body)

        if response.status_code == 200:
            print("\nPrompt response:")
            print(json.dumps(response.json(), indent=4))

            choices_data = track_semantic_scores(response.json())
            for choice in choices_data:
                df.loc[len(df)] = [choice[0], semantic_similarity, choice[1], choice[2], choice[3]]

            print("\nTop Implementation Tip:", choices_data[0][2])
            print("\nOther Tips:", *[choice[2] for choice in choices_data[1:]], sep="\n")
        else:
            print(f"\nError with prompt variation {variation}. Status code: {response.status_code}")
            print(json.dumps(response.json(), indent=4))

def fetch_data(endpoint, headers, success_message, fail_message):
    """Make a GET request and print the results."""
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        print(f"\n{success_message}:")
        print(json.dumps(response.json(), indent=4))
    else:
        print(f"\n{fail_message} {response.status_code}:")
        print(json.dumps(response.json(), indent=4))

def delete_data(endpoint, headers):
    """Make a DELETE request and print the results."""
    response = requests.delete(endpoint, headers=headers)
    if response.status_code == 200:
        print("\nOperation successful!")
    else:
        print(f"\nOperation failed with status code {response.status_code}:")
        print(json.dumps(response.json(), indent=4))

# --- MAIN EXECUTION ---

# 1. Log in to the service and update headers with the obtained token
token = login_to_promptmule()
headers["Authorization"] = f"Bearer {token}"
print("Login successful! Token acquired.")

# 2. Generate or retrieve the user's API key
api_key = generate_or_fetch_api_key()
print(f"API Key: {api_key}")

# 3. Gather user input for prompt testing
users_message = input(USERS_MESSAGE)

# 4. Define constants for prompt variations and requests
prompt_variations = 25
requests_per_variation = 5
df = pd.DataFrame(columns=['Prompt ID', 'Semantic Similarity', 'Score', 'Choice Content', 'Choice Index'])

# 5. Send prompt requests with varying semantic similarities and max response counts
semantic_nums = [1.0, 0.75, 0.5]
max_responses = list(range(1, 11))  # From 1 to 10
for sem_num in semantic_nums:
    send_prompt_requests(users_message, sem_num, random.choice(max_responses))

# 6. Display the results in a structured report
print("\nReport:")
print(df.to_string(index=False))

# 7. Fetch and display user data summary
fetch_data(ENDPOINT + LIST_API_KEYS, headers, "List of API keys", "Failed to list API keys")
fetch_data(ENDPOINT + 'usage', {"Authorization": f"Bearer {token}", "x-api-key": api_key}, "Usage Summary", "Failed to fetch usage summary")
fetch_data(ENDPOINT + 'usage/daily-stats', {"Authorization": f"Bearer {token}", "x-api-key": api_key}, "Daily Usage Stats", "Failed to fetch daily usage stats")

print("\nReport Summary:")
print(f"Username: {USERNAME}")
print(f"API Key: {api_key}")
print(f"Number of Cost-saving Strategy Variations Sent: {prompt_variations}")
print(f"Number of Requests per Strategy Variation: {requests_per_variation}")

# 8. Optionally allow user to delete their profile
#if input("\nDo you really want to delete your user profile? (yes/no): ").lower() == 'yes':
#    delete_data(ENDPOINT + DELETE, headers)
